# How to perform chemical equilibrium calculations

Reaktoro performs chemical equilibrium calculations by *minimizing the system's Gibbs energy* {cite}`Leal2017a`. In a Gibbs energy minimization problem, temperature and pressure are prescribed. The chemical system is considered **closed** (the mass of the chemical elements and the electrical charge are conserved in the reactive process). We give a simple example below in which we use class {{EquilibriumSolver}} to perform the chemical equilibrium calculation.

In [18]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

print("=== INITIAL STATE ===")
print(state)

solver = EquilibriumSolver(system)
solver.solve(state)  # compute the state of chemical equilibrium!

print("=== FINAL STATE ===")
print(state)

=== INITIAL STATE
+-----------------+---------+------+
| Property        |   Value | Unit |
+-----------------+---------+------+
| Temperature     | 1273.15 |    K |
| Pressure        |   1e+07 |   Pa |
| Charge:         |       0 |  mol |
| Element Amount: |         |      |
| :: H            |       4 |  mol |
| :: C            |       1 |  mol |
| :: O            |       1 |  mol |
| Species Amount: |         |      |
| :: CH4          |       1 |  mol |
| :: O2           |     0.5 |  mol |
| :: CO2          |   1e-16 |  mol |
| :: CO           |   1e-16 |  mol |
| :: H2O          |   1e-16 |  mol |
| :: H2           |   1e-16 |  mol |
+-----------------+---------+------+
=== FINAL STATE
+-----------------+-----------+------+
| Property        |     Value | Unit |
+-----------------+-----------+------+
| Temperature     |   1273.15 |    K |
| Pressure        |     1e+07 |   Pa |
| Charge:         |         0 |  mol |
| Element Amount: |           |      |
| :: H            |        

The final mole composition above was found by minimizing the Gibbs energy of the system. Note that the initial and final amounts of the elements H, C, and O are identical (the same would apply to the electric charge of the system if there were charged species in the problem).

However, there are many different classes of equilibrium problems where:

* **temperature and/or pressure may be unknown** (e.g., combustion in a rigid and adiabatic chamber where volume and internal energy are specified and temperature and pressure are calculated);
* **the chemical system is open to certain substances** (e.g., aqueous solution in equilibrium with a mixture of gases with known partial pressures or fugacities, such as the atmosphere).

Reaktoro's chemical equilibrium algorithm supports all these types of cases because it implements an algorithm to solve *parametric Gibbs energy minimization problems with general equilibrium constraints*. Thus, Reaktoro is not restricted to closed systems and prescribed conditions of temperature and pressure, as we will see in the next sections.

```{note}
If you've read the previous guides, you've probably already encountered the use of `equilibrate(state)` to bring `state`, an object of class {{ChemicalState}}, to a state of chemical equilibrium. This method exists for convenience and performs a Gibbs energy minimization on the entire chemical system. It is equivalent to using {{EquilibriumSolver}} as follows:

~~~py
solver = EquilibriumSolver(system)
solver.solve(state)
~~~
```

```{tip}
If you need many equilibrium calculations (e.g. in the context of reactive transport simulations), consider using the {{EquilibriumSolver}} class instead of the `equilibrate` method, as the latter creates an {{EquilibriumSolver}} object during each call without reuse afterwards, which can cause computation overhead.
```

## Calculation with given temperatures and pressures

## Calculation with given aqueous solution pH

Fixing the pH of an aqueous solution is a common procedure in biogeochemical modeling. This requires, however, the introduction of a new *degree of freedom* in the problem. This is because we cannot, in general, obtain a desired pH without titrating a substance in the solution. Thus, in the example given below, the system will be open to a substance: H<sup>+</sup>. *The unknown amount of H<sup>+</sup> necessary to enter or leave the system to attain the requested pH is the new degree of freedom*, and is calculated along with the amounts of species in equilibrium.

```{note}
The choice of H<sup>+</sup> as the *titrant* when fixing the pH of an aqueous solution may seem arbitrary and unintuitive to some coming from a different background or experienced with another convention used by other chemical modeling codes. For example, it can be questioned that a charged species not found isolated in nature (compared to HCl, for example) is chosen as the titrant. However, there are mathematical reasons behind this choice that make the problem to be solved more simply and efficiently. We will give examples later on how to combine this pH constraint with other conditions to demonstrate the full potential of this convention.
```

For our example, let's define a simple aqueous phase:

In [32]:
db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase("H2O H+ OH- Na+ Cl- HCO3- CO2 CO3-2")

system = ChemicalSystem(db, solution)

For this system, let's create a chemical state representing a 1 molal NaCl solution with 0.4 molal dissolved CO{{_2}}:

In [33]:
state = ChemicalState(system)
state.temperature(30.0, "celsius")
state.pressure(1.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.4, "mol")

## Calculation with given volume and internal energy

Let's consider again the combustion of 1 mol of CH{{_4}} and 0.5 mol of O{{_2}}. This time, however, we will not specify temperature and pressure, but volume and internal energy. This is to model the combustion of this gas mixture in a rigid and adiabatic chamber.

In [30]:
state0 = ChemicalState(system)
state0.temperature(25.0, "celsius")
state0.pressure(1.0, "bar")
state0.set("CH4", 1.0, "mol")
state0.set("O2",  0.5, "mol")

props0 = ChemicalProps(state0)

V0 = props0.volume()  # the initial volume of the gases
U0 = props0.internalEnergy()  # the initial internal energy of the gases

In [31]:

specs = EquilibriumSpecs(system)
specs.volume()
specs.internalEnergy()

conditions = EquilibriumConditions(specs)
conditions.volume(V0)
conditions.internalEnergy(U0)

conditions.setLowerBoundPressure(1.0, "bar")
conditions.setLowerBoundTemperature(25.0, "celsius")

state = ChemicalState(state0)

solver = EquilibriumSolver(specs)
solver.solve(state, conditions)

print(state)

+-----------------+-----------+------+
| Property        |     Value | Unit |
+-----------------+-----------+------+
| Temperature     |   1230.15 |    K |
| Pressure        |    768530 |   Pa |
| Charge:         |         0 |  mol |
| Element Amount: |           |      |
| :: H            |         4 |  mol |
| :: C            |         1 |  mol |
| :: O            |         1 |  mol |
| Species Amount: |           |      |
| :: CH4          |  0.102994 |  mol |
| :: O2           |     1e-16 |  mol |
| :: CO2          | 0.0261655 |  mol |
| :: CO           |   0.87084 |  mol |
| :: H2O          | 0.0768286 |  mol |
| :: H2           |   1.71718 |  mol |
+-----------------+-----------+------+





Let's see different use cases for {{EquilibriumSolver}} in the next sections.

## Checking if the calculation was successful

Performing chemical equilibrium calculations is not a trivial task. It involves highly complicated algorithms to solve different mathematical problems (eg non-linear equations, matrix equations). Therefore, it is possible that a calculation could fail. This can happen due to many factors. However, the most common are:

* the formulation of the equilibrium problem is ill-formed (e.g. specifying a set of conflicting equilibrium constraints that cannot be achieved chemically/thermodynamically)
* the use of poor initial guesses.

So you might want to check at the end of each equilibrium calculation if it was successful:

In [13]:
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

solver = EquilibriumSolver(system)
result = solver.solve(state)

print("Succeesful?", result.optima.succeeded)

Succeesful? True


The object `result` above is of type {{EquilibriumResult}} and it also contains other details about the calculation, such as the number of iterations needed for convergence:

In [12]:
print("Iterations:", result.optima.iterations)

Iterations: 27


## Calculations with given initial element amounts